<a href="https://colab.research.google.com/github/bmenderes/ArizaAnalizi/blob/main/MakineArizaAnlizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: 'Zaman': 2023-11-20 ile 2023-11-27 tarihleri arasında rastgele 300 zaman noktası içeren bir zaman serisi
# 'Vardiya': 1 ile 3 arasında rastgele vardiya numaraları
# 'Durus_Suresi': 5 ile 120 dakika arasında rastgele duruş süresi
# 'Ariza_Turu': 'mekanik', 'elektrik' veya 'hava' kategorilerinden rastgele seçilen arıza türleri
# 'Ariza_Alt_Turu': 'parca asinmasi', 'kirilma', 'gevseme', 'sensor', 'kablo', 'haberlesme' kategorilerinden rastgele bir alt arıza türü
# 'Etkilenen_Bolum': 1 ile 5 arasında rastgele etkilenen bölüm numaraları
# Veri çerçevesini oluşturup bir makine_ariza_veri_seti.csv dosyasına kaydedecek bir Python kodu yazar mısın ?

import pandas as pd
import numpy as np
import random

# Tarih aralığı
start_date = pd.to_datetime('2023-11-20')
end_date = pd.to_datetime('2023-11-27')

# Zaman noktaları
time_points = pd.date_range(start=start_date, end=end_date, periods=300)

# Veri çerçevesi oluşturma
data = {
    'Zaman': time_points,
    'Vardiya': np.random.randint(1, 4, size=300),
    'Durus_Suresi': np.random.randint(5, 121, size=300),
    'Ariza_Turu': np.random.choice(['mekanik', 'elektrik', 'hava'], size=300),
    'Ariza_Alt_Turu': np.random.choice(['parca asinmasi', 'kirilma', 'gevseme', 'sensor', 'kablo', 'haberlesme'], size=300),
    'Etkilenen_Bolum': np.random.randint(1, 6, size=300)
}

df = pd.DataFrame(data)

# CSV dosyasına kaydetme
df.to_csv('makine_ariza_veri_seti.csv', index=False)

# Dosyayı indirme
from google.colab import files
files.download('makine_ariza_veri_seti.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: Root Mean Squared Error
# Veri setindeki ortalama duruş süresi
# Mean Absolute Error (MAE)
# Örnek bir veri için rasgele bir (np.random.seed) veri için tahmin edilen duruş zamanı(tarih formatında tahminleme veri setinde yer alan son tarihten sonrası olacak şekilde),
# vardiyası, duruş süresi, arıza türü, ariza alt türü, ve etkilenen bölümü görüntüleyebilecegim bir kod oluşturur musun

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Örnek veri (önceki koddan oluşturulan CSV dosyasını yükleyin)
df = pd.read_csv('makine_ariza_veri_seti.csv')

# Ortalama duruş süresi
ortalama_durus_suresi = df['Durus_Suresi'].mean()
print(f"Ortalama Duruş Süresi: {ortalama_durus_suresi}")

# RMSE ve MAE hesaplama için örnek bir tahmin oluşturma
np.random.seed(42)  # Rastgele sayı üretecinin başlangıç değerini ayarla
df['Tahmin_Edilen_Durus_Suresi'] = np.random.randint(5, 121, size=len(df))

rmse = np.sqrt(mean_squared_error(df['Durus_Suresi'], df['Tahmin_Edilen_Durus_Suresi']))
mae = mean_absolute_error(df['Durus_Suresi'], df['Tahmin_Edilen_Durus_Suresi'])

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


# Tahmin edilen duruş zamanı (son tarihten sonraki bir tarih)
last_date = pd.to_datetime(df['Zaman'].max())
tahmin_tarihi = last_date + pd.DateOffset(days=1)

# Örnek bir veri için bilgiler
ornek_veri_index = 0  # Örnek olarak ilk satırı alıyoruz. İstediğiniz bir satırı seçebilirsiniz.
ornek_veri = df.iloc[ornek_veri_index]

print("\nÖrnek Veri:")
print(f"Tahmin Tarihi: {tahmin_tarihi.strftime('%Y-%m-%d')}")
print(f"Vardiya: {ornek_veri['Vardiya']}")
print(f"Duruş Süresi: {ornek_veri['Durus_Suresi']}")
print(f"Arıza Türü: {ornek_veri['Ariza_Turu']}")
print(f"Arıza Alt Türü: {ornek_veri['Ariza_Alt_Turu']}")
print(f"Etkilenen Bölüm: {ornek_veri['Etkilenen_Bolum']}")


Ortalama Duruş Süresi: 62.48
RMSE: 47.439997189432184
MAE: 39.346666666666664

Örnek Veri:
Tahmin Tarihi: 2023-11-28
Vardiya: 3
Duruş Süresi: 90
Arıza Türü: elektrik
Arıza Alt Türü: kablo
Etkilenen Bölüm: 5


In [4]:
# prompt: şimdi makine öğrenmesi kullanarak bu tahminlemeyi yapabilecegim Random Forest Regressor modelini kullanarak önce eğitip daha sonra aşağıdaki tahminlemeleri yapar mısın ?
# Ayrıca sonuçlar ekranında numerik olarak verilmiş ifadelerin kategorik karşılıklarının olduğu bir tablo da hazirlar misin ? Lütfen önce kodu kendi içerisinde çalıştırıp doğruluğunu kontrol et.
# Root Mean Squared Error
# Veri setindeki ortalama duruş süresi
# Mean Absolute Error (MAE)
# Örnek bir veri için tahmin edilen duruş zamanı(tarih formatında tahminleme veri setinde yer alan son tarihten sonrası olacak şekilde),
# vardiyası, duruş süresi, arıza türü, ariza alt türü, ve etkilenen bölümü görüntüleyebilecegim bir kod oluşturur musun

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# Veri setini yükle
df = pd.read_csv('makine_ariza_veri_seti.csv')

# Tarih sütununu datetime tipine dönüştür
df['Zaman'] = pd.to_datetime(df['Zaman'])

# Kategorik değişkenleri sayısal hale dönüştürme
label_encoders = {}
for column in ['Ariza_Turu', 'Ariza_Alt_Turu']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Özellik ve hedef değişkenleri ayırma
X = df[['Vardiya', 'Ariza_Turu', 'Ariza_Alt_Turu', 'Etkilenen_Bolum']]
y = df['Durus_Suresi']

# Veri setini eğitim ve test kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Regressor modelini eğitme
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Tahminler yapma
y_pred = rf_model.predict(X_test)

# Performans ölçümleri
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"Ortalama Duruş Süresi: {df['Durus_Suresi'].mean()}")

# Örnek bir veri için tahmin
ornek_veri = pd.DataFrame({'Vardiya': [3], 'Ariza_Turu': [label_encoders['Ariza_Turu'].transform(['mekanik'])[0]], 'Ariza_Alt_Turu': [label_encoders['Ariza_Alt_Turu'].transform(['parca asinmasi'])[0]], 'Etkilenen_Bolum': [2]})
tahmin_edilen_durus_suresi = rf_model.predict(ornek_veri)[0]
last_date = pd.to_datetime(df['Zaman'].max())
tahmin_tarihi = last_date + pd.DateOffset(days=1)

print(f"\nÖrnek Veri için Tahmin Edilen Duruş Süresi: {tahmin_edilen_durus_suresi}")
print(f"Tahmin Tarihi: {tahmin_tarihi.strftime('%Y-%m-%d')}")
print(f"Vardiya: 3")
print(f"Arıza Türü: mekanik")
print(f"Arıza Alt Türü: parca asinmasi")
print(f"Etkilenen Bölüm: 2")


# Kategorik Değişken Karşılıkları
print("\nKategorik Değişken Karşılıkları:")
for column, le in label_encoders.items():
    print(f"\n{column}:")
    print(pd.DataFrame({'Sayısal': le.transform(le.classes_), 'Kategorik': le.classes_}))


RMSE: 38.503367702417044
MAE: 31.824727489177487
Ortalama Duruş Süresi: 61.20333333333333

Örnek Veri için Tahmin Edilen Duruş Süresi: 36.007000000000005
Tahmin Tarihi: 2023-11-28
Vardiya: 3
Arıza Türü: mekanik
Arıza Alt Türü: parca asinmasi
Etkilenen Bölüm: 2

Kategorik Değişken Karşılıkları:

Ariza_Turu:
   Sayısal Kategorik
0        0  elektrik
1        1      hava
2        2   mekanik

Ariza_Alt_Turu:
   Sayısal       Kategorik
0        0         gevseme
1        1      haberlesme
2        2           kablo
3        3         kirilma
4        4  parca asinmasi
5        5          sensor


In [7]:
import pandas as pd

# Veri setini yükleme
file_path = "makine_ariza_veri_seti.csv"  # Dosya adınızı buraya yazın
df = pd.read_csv(file_path)

# "mekanik" ve "parca asinmasi" filtreleme
mekanik_parca_asinmasi = df[
    (df["Ariza_Turu"] == "mekanik") & (df["Ariza_Alt_Turu"] == "parca asinmasi")
]
mekanik_arizalar = df[df["Ariza_Turu"] == "mekanik"]

# Ortalama duruş süresi hesaplama
ortalama_durus_suresi1 = mekanik_parca_asinmasi["Durus_Suresi"].mean()
ortalama_durus_suresi2 = mekanik_arizalar["Durus_Suresi"].mean()

print("Mekanik parça aşınması kaynaklı ortalama duruş süresi:", ortalama_durus_suresi1)
print("Mekanik arızalar kaynaklı ortalama duruş süresi:", ortalama_durus_suresi2)

Mekanik parça aşınması kaynaklı ortalama duruş süresi: 56.25
Mekanik arızalar kaynaklı ortalama duruş süresi: 58.663716814159294
